# Abstract

This is a clone of the script at https://www.kaggle.com/ceshine/lgbm-starter which is intended to give an idea of how to structure the data for trainig

# Prelude 

## Configuration

In [40]:
DataSetPath = "/home/bryanfeeney/Workspace/OttomanDiviner/favorita/"

StoresPath   = DataSetPath + "stores.csv.gz"
ItemsPath    = DataSetPath + "items.csv.gz"
OilPricePath = DataSetPath + "oil.csv.gz"
HolidaysPath = DataSetPath + "holidays_events.csv.gz"
Transactions = DataSetPath + "transactions.csv.gz"
TrainData    = DataSetPath + "train-2017.csv.gz"
TestData     = DataSetPath + "test.csv.gz"
# TrainData    = DataSetPath + "train-2018.csv.gz"
# TestData     = DataSetPath + "query-2018.csv"

FutureDaysToCalculate=16
WeeksOfHistoryForFeature=8
WeeksOfHistoryForFeatureOnValidation=3

## Imports

In [41]:
from datetime import date, datetime, timedelta

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

# Intro to the Data

In [79]:
cumul_sales = pd.read_csv(
    TrainData, 
    usecols=[1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    converters={'unit_sales': lambda u: np.log1p(float(u)) if float(u) > 0 else 0},
    parse_dates=["date"],
    compression='gzip'
)


In [80]:

cumul_sales_query = pd.read_csv(
    TestData,
    usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
)

In [81]:
query_start_date = str(cumul_sales_query.iloc[0,1]).split(" ")[0]

In [82]:
query_start_date

'2017-08-16'

In [83]:
cumul_sales_query = cumul_sales_query.set_index(
    ['store_nbr', 'item_nbr', 'date']
)

In [84]:
cumul_sales.shape, cumul_sales_query.shape

((23808260, 5), (3370464, 2))

In [88]:
cumul_sales.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion
0,2017-01-01,25,103665,2.079442,False
1,2017-01-01,25,105574,0.693147,False
2,2017-01-01,25,105857,1.609438,False
3,2017-01-01,25,106716,1.098612,False
4,2017-01-01,25,108698,1.098612,False


<font color="red">DEBUG</font>

In [86]:
cumul_sales_query.head()

id  onpromotion
store_nbr item_nbr date                              
1         96995    2017-08-16  125497040        False
          99197    2017-08-16  125497041        False
          103501   2017-08-16  125497042        False
          103520   2017-08-16  125497043        False
          103665   2017-08-16  125497044        False

In [73]:
promo_variables_test = cumul_sales_query[["onpromotion"]].unstack(level=-1).fillna(False)

In [74]:
promo_variables_test.head()

onpromotion                                              \
date                2017-08-16 2017-08-17 2017-08-18 2017-08-19 2017-08-20   
store_nbr item_nbr                                                           
1         96995          False      False      False      False      False   
          99197          False      False      False      False      False   
          103501         False      False      False      False      False   
          103520         False      False      False      False      False   
          103665         False      False      False      False      False   

                                                                           \
date               2017-08-21 2017-08-22 2017-08-23 2017-08-24 2017-08-25   
store_nbr item_nbr                                                          
1         96995         False      False      False      False      False   
          99197         False      False      False      False      False   
          103501        False      False      False      False      False   
          103520        False      False      False      False      False   
          103665        False      False      False      False      False   

                                                                           \
date               2017-08-26 2017-08-27 2017-08-28 2017-08-29 2017-08-30   
store_nbr item_nbr                                                          
1         96995         False      False      False      False      False   
          99197         False      False      False      False      False   
          103501        False      False      False      False      False   
          103520        False      False      False      False      False   
          103665        False      False      False      False      False   

                               
date               2017-08-31  
store_nbr item_nbr             
1         96995         False  
          99197         False  
          103501        False  
          103520        False  
          103665        False

<font color=red>DEBUG</font>

In [77]:
cumul_sales

,date
store_nbr,item_nbr


In [50]:
cumul_sales_query.iloc[0,:]

id             125497040
onpromotion        False
Name: (1, 96995, 2017-08-16 00:00:00), dtype: object

In [51]:
items = pd.read_csv(
    ItemsPath,
).set_index("item_nbr")

stores = pd.read_csv(
    StoresPath
).set_index("store_nbr")

In [52]:
cumul_sales_query

id  onpromotion
store_nbr item_nbr date                              
1         96995    2017-08-16  125497040        False
          99197    2017-08-16  125497041        False
          103501   2017-08-16  125497042        False
          103520   2017-08-16  125497043        False
          103665   2017-08-16  125497044        False
          105574   2017-08-16  125497045        False
          105575   2017-08-16  125497046        False
          105576   2017-08-16  125497047        False
          105577   2017-08-16  125497048        False
          105693   2017-08-16  125497049        False
          105737   2017-08-16  125497050        False
          105857   2017-08-16  125497051        False
          106716   2017-08-16  125497052        False
          108079   2017-08-16  125497053        False
          108634   2017-08-16  125497054        False
          108696   2017-08-16  125497055        False
          108698   2017-08-16  125497056        False
          108701   2017-08-16  125497057         True
          108786   2017-08-16  125497058        False
          108797   2017-08-16  125497059         True
          108831   2017-08-16  125497060        False
          108833   2017-08-16  125497061        False
          108862   2017-08-16  125497062        False
          108952   2017-08-16  125497063        False
          111223   2017-08-16  125497064        False
          111397   2017-08-16  125497065        False
          112830   2017-08-16  125497066        False
          114778   2017-08-16  125497067        False
          114790   2017-08-16  125497068         True
          114799   2017-08-16  125497069        False
...                                  ...          ...
54        2127921  2017-08-31  128867474        False
          2127992  2017-08-31  128867475        False
          2128628  2017-08-31  128867476        False
          2128799  2017-08-31  128867477        False
          2129334  2017-08-31  128867478        False
          2129350  2017-08-31  128867479        False
          2129387  2017-08-31  128867480        False
          2129515  2017-08-31  128867481        False
          2129616  2017-08-31  128867482        False
          2129678  2017-08-31  128867483        False
          2129786  2017-08-31  128867484        False
          2129790  2017-08-31  128867485        False
          2129892  2017-08-31  128867486        False
          2129994  2017-08-31  128867487        False
          2130131  2017-08-31  128867488        False
          2130219  2017-08-31  128867489        False
          2130265  2017-08-31  128867490        False
          2130352  2017-08-31  128867491        False
          2130474  2017-08-31  128867492        False
          2130521  2017-08-31  128867493        False
          2130526  2017-08-31  128867494        False
          2130553  2017-08-31  128867495        False
          2131010  2017-08-31  128867496        False
          2131572  2017-08-31  128867497        False
          2131699  2017-08-31  128867498        False
          2132163  2017-08-31  128867499        False
          2132318  2017-08-31  128867500        False
          2132945  2017-08-31  128867501        False
          2132957  2017-08-31  128867502        False
          2134244  2017-08-31  128867503        False

[3370464 rows x 2 columns]

In [53]:
cumul_sales.shape

(23808260, 5)

In [54]:
cumul_sales_query.shape

(3370464, 2)

In [55]:
items.shape

(4100, 3)

## Select only Last Three Months

This is a peculiar one, and it **games the benchmark** in a not great way. Essentially it uses the last 11 weeks of data before the prediction threshold to predict what's happening next

In [56]:
nowtime = datetime.now()
now = date(nowtime.year, nowtime.month, nowtime.day)

# How far back to go to start generating trend features for demand
data_start             = now - timedelta(7*11) + timedelta(1)
training_history_start = now - timedelta(7*WeeksOfHistoryForFeature) + timedelta(1)
validation_start       = now - timedelta(7*WeeksOfHistoryForFeatureOnValidation) + timedelta(1)



In [57]:
data_start, training_history_start, query_start_date

(datetime.date(2018, 3, 13), datetime.date(2018, 4, 3), '2017-08-16')

In [58]:
cumul_sales = cumul_sales[cumul_sales.date.isin(
    pd.date_range(data_start, periods=7 * 11))].copy()


In [59]:
cumul_sales.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion


In [60]:
cumul_sales.shape

(0, 5)

In [61]:
cumul_sales.iloc[-1,:]

IndexError: single positional indexer is out-of-bounds

## Creating Promotion Variables

So this is a tricky. If one presumes that on-promotion will lead to a boost in demand, if if we presume we'll know *whats on promotion in advance*, then we can create variables to say that this product will be on promotion 1, 2, 3, ... 16 days from now (16 days in the future is the target)

In this case, this is also peculiar, there is a column for every single day!

In [62]:
promo_variables = cumul_sales.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]]

In [63]:
promo_variables.head()

,,,onpromotion
store_nbr,item_nbr,date,


In [64]:
promo_variables = cumul_sales.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(
        level=-1).fillna(False)



In [65]:
promo_variables.head()

store_nbr,item_nbr


In [66]:
promo_variables.columns = promo_variables.columns.get_level_values(1)

promo_variables_query = cumul_sales_query[["onpromotion"]].unstack(level=-1).fillna(False)
promo_variables_query.columns = promo_variables_query.columns.get_level_values(1)
promo_variables_query = promo_variables_query.reindex(promo_variables.index).fillna(False)

promo_variables_train_and_query = pd.concat([promo_variables, promo_variables_query], axis=1)


In [67]:
promo_variables.shape, items.shape[0] * stores.shape[0]

((0, 0), 221400)

In [68]:
cumul_sales.shape, cumul_sales_query.shape

((0, 5), (3370464, 2))

#  Unstack unit sales - do it across all days in a sliding window

Ah... they're creating a multi-task learning problem

In [69]:
cumul_sales = cumul_sales.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(
        level=-1).fillna(0)
cumul_sales.columns = cumul_sales.columns.get_level_values(1)
cumul_sales.shape

(0, 0)

In [70]:
cumul_sales.head()

,date
store_nbr,item_nbr


## Make items match other data frames

They're sacraficing generability

In [71]:
items = items.reindex(cumul_sales.index.get_level_values(1))
items.head()

,family,class,perishable
item_nbr,,,


In [72]:
items.shape

(0, 3)

## Time futzing

In [ ]:
# Return that portion of the data frame that corresponds to the time period
#   beginning "minus" days before "dt" and extending for "periods" days
def get_timespan(df, dt, minus, periods):
    return df[
        pd.date_range(dt - timedelta(days=minus), periods=periods)
    ]

In [ ]:
def prepare_dataset(cumul_sales, promo_variables_train_and_query, start_date, is_train=True):
    X = pd.DataFrame({  # Mean target for different retrospective timespans & total # promotions
        "mean_3_2017": get_timespan(cumul_sales, start_date, 3, 3).mean(axis=1).values,
        "mean_7_2017": get_timespan(cumul_sales, start_date, 7, 7).mean(axis=1).values,
        "mean_14_2017": get_timespan(cumul_sales, start_date, 14, 14).mean(axis=1).values,
        "promo_14_2017": get_timespan(promo_variables_train_and_query, start_date, 14, 14).sum(axis=1).values
    })
    for i in range(16):  # Promotions on future days
        X["promo_{}".format(i)] = promo_variables_train_and_query[
            start_date + timedelta(days=i)].values.astype(np.uint8)
    if is_train:
        y = cumul_sales[  # Target values for future days
            pd.date_range(start_date, periods=16)
        ].values
        return X, y
    return X

In [ ]:
promo_variables_train_and_query.shape

In [ ]:
training_history_start, validation_start, now

In [ ]:
promo_variables

In [ ]:
print("Preparing dataset...")
X_l, y_l = [], []
for i in range(4):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(cumul_sales, promo_variables_train_and_query, training_history_start + delta)
    X_l.append(X_tmp)
    y_l.append(y_tmp)
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)
del X_l, y_l

X_validate, y_validate = prepare_dataset(cumul_sales, promo_variables_train_and_query, validation_start)

X_query = prepare_dataset(cumul_sales, promo_variables_train_and_query, now, is_train=False)

In [ ]:
X_train.shape, X_validate.shape, X_query.shape

This dataset is **super gamey**. They're using the means for the week, fortnight, and last three days, and then seeing how to permute it to generate values for the following window of time. It's hardcoded to product IDs, not categories.

It does however, permit multi-task learning, and therefore better representation learning

It does not incorporate any information about seasonality at all, and so would fall arse over face at Christmas



In [ ]:
print("Training and predicting models...")
params = {
    'num_leaves': 2**5 - 1,
    'objective': 'regression_l2',
    'max_depth': 8,
    'min_data_in_leaf': 50,
    'learning_rate': 0.05,
    'feature_fraction': 0.75,
    'bagging_fraction': 0.75,
    'bagging_freq': 1,
    'metric': 'l2',
    'num_threads': 4
}

In [ ]:
MAX_ROUNDS = 1000
validate_pred = []
query_pred = []
cate_vars = []
for i in range(16):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    
    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i],
        categorical_feature=cate_vars,
        weight=pd.concat([items["perishable"]] * 4) * 0.25 + 1
    )
    
    dvalidate = lgb.Dataset(
        X_validate, label=y_validate[:, i], reference=dtrain,
        weight=items["perishable"] * 0.25 + 1,
        categorical_feature=cate_vars)
    
    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS,
        valid_sets=[dtrain, dvalidate], early_stopping_rounds=50, verbose_eval=50
    )
    
    print("\n".join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True
    )))
    
    
    validate_pred.append(bst.predict(
        X_validate, num_iteration=bst.best_iteration or MAX_ROUNDS))
    
    query_pred.append(bst.predict(
        X_query, num_iteration=bst.best_iteration or MAX_ROUNDS))

In [ ]:
print("Validation mse:", np.sqrt(mean_squared_error(
    np.expm1(y_validate), np.expm1(np.array(validate_pred)).transpose())))

In [ ]:
validate_pred

In [ ]:
query_pred

In [ ]:
print("Making submission...")
y_query = np.array(query_pred).transpose()
df_preds = pd.DataFrame(
    y_query, index=cumul_sales.index,
    columns=pd.date_range(query_start_date, periods=16)
).stack().to_frame("unit_sales")
df_preds.to_csv("/tmp/preds-2018.csv")


In [ ]:
df_preds

In [168]:
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

In [83]:

submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)

NameError: name 'df_test' is not defined

In [61]:
submission

id  unit_sales
store_nbr item_nbr date                             
1         96995    2017-08-16  125497040    0.273266
          99197    2017-08-16  125497041    0.225388
          103501   2017-08-16  125497042    0.000000
          103520   2017-08-16  125497043    0.648250
          103665   2017-08-16  125497044    1.352113
          105574   2017-08-16  125497045    3.222305
          105575   2017-08-16  125497046    7.469437
          105576   2017-08-16  125497047    0.000000
          105577   2017-08-16  125497048    0.294655
          105693   2017-08-16  125497049    0.319934
          105737   2017-08-16  125497050    0.763024
          105857   2017-08-16  125497051    3.247849
          106716   2017-08-16  125497052    1.781587
          108079   2017-08-16  125497053    0.346823
          108634   2017-08-16  125497054    0.033705
          108696   2017-08-16  125497055    1.357327
          108698   2017-08-16  125497056    0.585320
          108701   2017-08-16  125497057    1.804330
          108786   2017-08-16  125497058    1.744146
          108797   2017-08-16  125497059    2.840066
          108831   2017-08-16  125497060    0.000000
          108833   2017-08-16  125497061    0.000000
          108862   2017-08-16  125497062    0.704088
          108952   2017-08-16  125497063    1.000221
          111223   2017-08-16  125497064    3.262282
          111397   2017-08-16  125497065    0.431402
          112830   2017-08-16  125497066    1.090800
          114778   2017-08-16  125497067    1.617434
          114790   2017-08-16  125497068    2.928812
          114799   2017-08-16  125497069    0.731032
...                                  ...         ...
54        2127921  2017-08-31  128867474    0.000000
          2127992  2017-08-31  128867475    0.000000
          2128628  2017-08-31  128867476    0.000000
          2128799  2017-08-31  128867477    0.000000
          2129334  2017-08-31  128867478    0.000000
          2129350  2017-08-31  128867479    0.000000
          2129387  2017-08-31  128867480    0.000000
          2129515  2017-08-31  128867481    0.000000
          2129616  2017-08-31  128867482    0.000000
          2129678  2017-08-31  128867483    0.000000
          2129786  2017-08-31  128867484    0.000000
          2129790  2017-08-31  128867485    0.000000
          2129892  2017-08-31  128867486    0.000000
          2129994  2017-08-31  128867487    0.000000
          2130131  2017-08-31  128867488    0.000000
          2130219  2017-08-31  128867489    0.000000
          2130265  2017-08-31  128867490    0.000000
          2130352  2017-08-31  128867491    0.000000
          2130474  2017-08-31  128867492    0.000000
          2130521  2017-08-31  128867493    0.000000
          2130526  2017-08-31  128867494    0.000000
          2130553  2017-08-31  128867495    0.000000
          2131010  2017-08-31  128867496    0.000000
          2131572  2017-08-31  128867497    0.000000
          2131699  2017-08-31  128867498    0.000000
          2132163  2017-08-31  128867499    0.000000
          2132318  2017-08-31  128867500    0.000000
          2132945  2017-08-31  128867501    0.000000
          2132957  2017-08-31  128867502    0.000000
          2134244  2017-08-31  128867503    0.000000

[3370464 rows x 2 columns]

# Further Improvements

This is based on the work in this file: https://www.kaggle.com/vrtjso/lgbm-one-step-ahead

This was apparently in the top 10% at one point.

In [76]:
df_train = pd.read_csv(
    TrainData, usecols=[1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    converters={'unit_sales': lambda u: np.log1p(
        float(u)) if float(u) > 0 else 0},
    parse_dates=["date"],
    skiprows=range(1, 66458909)  # 2016-01-01
)

df_test = pd.read_csv(
    TestData, usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)

items = pd.read_csv(
    ItemsPath,
).set_index("item_nbr")

df_2017 = df_train.loc[df_train.date>=pd.datetime(2017,1,1)]
del df_train

promo_2017_train = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(
        level=-1).fillna(False)
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)
promo_2017_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)
promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)
del promo_2017_test, promo_2017_train

df_2017 = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(
        level=-1).fillna(0)
df_2017.columns = df_2017.columns.get_level_values(1)

items = items.reindex(df_2017.index.get_level_values(1))

def get_timespan(df, dt, minus, periods, freq='D'):
    return df[pd.date_range(dt - timedelta(days=minus), periods=periods, freq=freq)]

def prepare_dataset(t2017, is_train=True):
    X = pd.DataFrame({
        "day_1_2017": get_timespan(df_2017, t2017, 1, 1).values.ravel(),
        "mean_3_2017": get_timespan(df_2017, t2017, 3, 3).mean(axis=1).values,
        "mean_7_2017": get_timespan(df_2017, t2017, 7, 7).mean(axis=1).values,
        "mean_14_2017": get_timespan(df_2017, t2017, 14, 14).mean(axis=1).values,
        "mean_30_2017": get_timespan(df_2017, t2017, 30, 30).mean(axis=1).values,
        "mean_60_2017": get_timespan(df_2017, t2017, 60, 60).mean(axis=1).values,
        "mean_140_2017": get_timespan(df_2017, t2017, 140, 140).mean(axis=1).values,
        "promo_14_2017": get_timespan(promo_2017, t2017, 14, 14).sum(axis=1).values,
        "promo_60_2017": get_timespan(promo_2017, t2017, 60, 60).sum(axis=1).values,
        "promo_140_2017": get_timespan(promo_2017, t2017, 140, 140).sum(axis=1).values
    })
    for i in range(7):
        X['mean_4_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 28-i, 4, freq='7D').mean(axis=1).values
        X['mean_20_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 140-i, 20, freq='7D').mean(axis=1).values
    for i in range(16):
        X["promo_{}".format(i)] = promo_2017[
            t2017 + timedelta(days=i)].values.astype(np.uint8)
    if is_train:
        y = df_2017[
            pd.date_range(t2017, periods=16)
        ].values
        return X, y
    return X

print("Preparing dataset...")
t2017 = date(2017, 5, 31)
X_l, y_l = [], []
for i in range(6):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(
        t2017 + delta
    )
    X_l.append(X_tmp)
    y_l.append(y_tmp)
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)
del X_l, y_l
X_val, y_val = prepare_dataset(date(2017, 7, 26))
X_test = prepare_dataset(date(2017, 8, 16), is_train=False)

print("Training and predicting models...")
params = {
    'num_leaves': 31,
    'objective': 'regression',
    'min_data_in_leaf': 300,
    'learning_rate': 0.1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 2,
    'metric': 'l2',
    'num_threads': 4
}

MAX_ROUNDS = 500
val_pred = []
test_pred = []
cate_vars = []
for i in range(16):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i],
        categorical_feature=cate_vars,
        weight=pd.concat([items["perishable"]] * 6) * 0.25 + 1
    )
    dval = lgb.Dataset(
        X_val, label=y_val[:, i], reference=dtrain,
        weight=items["perishable"] * 0.25 + 1,
        categorical_feature=cate_vars)
    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS,
        valid_sets=[dtrain, dval], early_stopping_rounds=50, verbose_eval=100
    )
    print("\n".join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True
    )))
    val_pred.append(bst.predict(
        X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    test_pred.append(bst.predict(
        X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))

print("Validation mse:", mean_squared_error(
    y_val, np.array(val_pred).transpose()))

print("Making submission...")
y_test = np.array(test_pred).transpose()
df_preds = pd.DataFrame(
    y_test, index=df_2017.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
submission.to_csv('lgb.csv', float_format='%.4f', index=None)


Preparing dataset...
Training and predicting models...
Step 1


/home/bryanfeeney/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1036: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 50 rounds.
[100]	training's l2: 0.30191	valid_1's l2: 0.29409
[200]	training's l2: 0.298363	valid_1's l2: 0.292741
[300]	training's l2: 0.295918	valid_1's l2: 0.292337
[400]	training's l2: 0.293791	valid_1's l2: 0.2921
[500]	training's l2: 0.29199	valid_1's l2: 0.29195
Did not meet early stopping. Best iteration is:
[500]	training's l2: 0.29199	valid_1's l2: 0.29195
mean_7_2017: 1882639.38
mean_14_2017: 1229821.77
promo_0: 104143.51
day_1_2017: 89857.46
mean_20_dow0_2017: 84245.48
mean_3_2017: 76646.29
mean_30_2017: 76583.57
mean_4_dow0_2017: 58919.38
mean_60_2017: 33035.18
promo_14_2017: 28619.72
promo_7: 9432.45
mean_4_dow5_2017: 7417.05
mean_140_2017: 7406.32
promo_60_2017: 6740.72
mean_20_dow4_2017: 5611.55
promo_140_2017: 5493.72
mean_4_dow6_2017: 4633.44
mean_4_dow2_2017: 3813.74
mean_20_dow2_2017: 3343.78
mean_4_dow3_2017: 2824.66
promo_9: 2814.25
mean_4_dow1_2017: 2707.00
mean_20_dow3_2017: 2642.58
mean_20_dow1_2017: 2616.99
me

Step 7
Training until validation scores don't improve for 50 rounds.
[100]	training's l2: 0.346219	valid_1's l2: 0.421254
[200]	training's l2: 0.341874	valid_1's l2: 0.420923
[300]	training's l2: 0.339035	valid_1's l2: 0.420617
[400]	training's l2: 0.336652	valid_1's l2: 0.420437
Early stopping, best iteration is:
[393]	training's l2: 0.336799	valid_1's l2: 0.420378
mean_14_2017: 1274004.42
mean_30_2017: 842088.63
mean_7_2017: 445757.89
mean_20_dow6_2017: 152802.16
mean_3_2017: 145241.24
promo_6: 128192.88
mean_4_dow6_2017: 127503.94
mean_60_2017: 123326.90
promo_14_2017: 21944.55
day_1_2017: 13872.06
promo_3: 11205.46
promo_7: 9013.03
mean_4_dow5_2017: 8533.91
mean_20_dow5_2017: 8280.78
promo_60_2017: 8112.57
promo_140_2017: 6018.04
mean_20_dow1_2017: 4884.13
promo_5: 4487.41
mean_140_2017: 4382.31
promo_13: 3877.01
mean_4_dow0_2017: 3650.61
mean_20_dow0_2017: 3639.82
mean_20_dow3_2017: 3507.37
mean_4_dow1_2017: 3355.21
mean_20_dow4_2017: 2943.12
promo_4: 2784.87
promo_0: 2663.62
mean

[200]	training's l2: 0.365615	valid_1's l2: 0.376333
[300]	training's l2: 0.362327	valid_1's l2: 0.376258
Early stopping, best iteration is:
[338]	training's l2: 0.361236	valid_1's l2: 0.37609
mean_30_2017: 1601076.04
mean_14_2017: 588977.50
mean_60_2017: 374936.35
mean_7_2017: 305452.67
mean_3_2017: 148957.18
promo_12: 93518.03
mean_20_dow5_2017: 85159.77
mean_4_dow5_2017: 71638.34
promo_13: 19595.91
promo_14_2017: 16011.22
promo_14: 13355.62
promo_10: 11265.36
mean_140_2017: 9927.43
promo_60_2017: 8798.33
day_1_2017: 7950.62
mean_20_dow0_2017: 7544.02
promo_140_2017: 6088.78
mean_20_dow6_2017: 5888.75
mean_4_dow6_2017: 4725.31
promo_11: 4479.14
mean_4_dow0_2017: 3756.38
mean_20_dow3_2017: 3664.43
promo_9: 3613.91
mean_20_dow2_2017: 3341.99
mean_4_dow2_2017: 2714.48
mean_20_dow1_2017: 2701.22
mean_4_dow3_2017: 2566.89
mean_20_dow4_2017: 2531.39
promo_15: 2495.30
mean_4_dow1_2017: 2301.79
mean_4_dow4_2017: 2274.90
promo_7: 2189.77
promo_0: 1734.63
promo_8: 1699.81
promo_5: 1614.77
prom

In [81]:
print("Validation mse:", mean_squared_error(
    np.expm1(y_validate), np.expm1(np.array(validate_pred)).transpose()))

Validation mse: 275.7862813287468


In [78]:
np.sqrt(275), np.sqrt(247)

(16.583123951777, 15.716233645501712)